In [1]:
import pandas as pd

# = = = = = = = = = = = = = = =

from urllib.parse import urlparse, parse_qs, urlencode

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

import json

from bs4 import BeautifulSoup
from lxml import etree

In [2]:
list_series_error, df_input = [], pd.read_excel('./file/input.xlsx',
                                                header=0,
                                                dtype=str).fillna('')

df_input['No'] = df_input['No'].astype(int)

count_remain = len(df_input)

print(f'总数量：{count_remain}')
print()
df_input

总数量：2



,No,Url
0,1,https://www.ebay.com/sch/i.html?_dkr=1&iconV2R...
1,2,https://www.ebay.de/sch/i.html?_dkr=1&iconV2Re...


In [3]:
(crawler_index, crawler_series) = next(df_input.iterrows())

crawler_index, crawler_series

(0,
 No                                                     1
 Url    https://www.ebay.com/sch/i.html?_dkr=1&iconV2R...
 Name: 0, dtype: object)

In [4]:
import time

url_parse = urlparse(crawler_series['Url'])

dict_param = parse_qs(url_parse.query)
dict_param.pop('_pgn', None)
dict_param['orig_cvip'] = 'true'
dict_param['_ipg'] = '240'
if crawler_series['Url'].startswith('https://www.ebay.com/'):
    dict_param['_ul'] = 'US'
    dict_param['_stpos'] = '91710'
elif crawler_series['Url'].startswith('https://www.ebay.de/'):
    dict_param['_ul'] = 'DE'
    dict_param['_stpos'] = '10115'
elif crawler_series['Url'].startswith('https://www.ebay.co.uk/'):
    dict_param['_ul'] = 'GB'
    dict_param['_stpos'] = 'BS81QU'
elif crawler_series['Url'].startswith('https://www.ebay.com.au/'):
    dict_param['_ul'] = 'AU'
    dict_param['_stpos'] = '2019'
elif crawler_series['Url'].startswith('https://www.ebay.ca/'):
    dict_param['_ul'] = 'CA'
    dict_param['_stpos'] = 'M5S2E8'

url_base = f'{url_parse.scheme}://{url_parse.netloc}{url_parse.path}?{urlencode(dict_param, doseq=True)}'

list_item_number = []
for count_cycle in range(10):

    page = 1
    while True:

        count_retry = 0
        while True:
            count_retry += 1
            print(count_cycle + 1, page, count_retry)
            try:
                url_request = f'{url_base}&_pgn={page}'

                resp = requests.get(url_request,
                                    headers=get_header(),
                                    proxies=get_proxy(),
                                    timeout=(5, 15))

                if resp.status_code == 200:
                    soup = BeautifulSoup(resp.text, 'lxml')
                    html = etree.HTML(str(soup))

                    dict_ = json.loads(resp.text.split('$MC=(window.$MC||[]).concat(')[1].split(']}]})</script>')[0].strip() + ']}]}')

                    list_item_number_temp = [w[2]['model']['action']['params']['itemId'].strip() for w in dict_['o']['w'] if type(w[2]) == dict and 'model' in w[2] and 'action' in w[2]['model'] and 'params' in w[2]['model']['action'] and 'itemId' in w[2]['model']['action']['params']]
                    if list_item_number_temp:
                        break
            except KeyboardInterrupt:
                break
            except:
                time.sleep(3)
                continue

        # = = = = = = = = = = = = = = =

        list_item_number.extend(list_item_number_temp)
        list_item_number = list(dict.fromkeys(list_item_number))

        # = = = = = = = = = = = = = = =

        print(f' - {len(list_item_number)}')

        # = = = = = = = = = = = = = = =

        page += 1

        # = = = = = = = = = = = = = = =

        if not html.xpath('//nav[@class="pagination"]/a[@type="next"]'):
            break

list_item_number_temp

1 1 1
 - 240
1 2 1
 - 453
1 3 1
 - 465
2 1 1
 - 472
2 2 1
 - 473
2 3 1
2 3 2
 - 476
3 1 1
3 1 2
 - 480
3 2 1
 - 480
3 3 1
 - 496
4 1 1
 - 496
4 2 1
 - 496
4 3 1
 - 496
5 1 1
 - 496
5 2 1
 - 496
5 3 1
5 3 2
 - 496
6 1 1
 - 512
6 2 1
 - 512
6 3 1
 - 512
7 1 1
 - 512
7 2 1
 - 512
7 3 1
 - 512
8 1 1
 - 512
8 2 1
 - 512
8 3 1
 - 512
9 1 1
 - 512
9 2 1
 - 512
9 3 1
9 3 2
 - 512
10 1 1
10 1 2
 - 512
10 2 1
 - 512
10 3 1
 - 512


['335511415366',
 '335597310497',
 '335597260874',
 '235648004912',
 '335464200802',
 '256646745710',
 '335524284452',
 '235574997420',
 '335511444096',
 '335597279925',
 '235648003081',
 '256580274981',
 '235741776768',
 '256566855836',
 '335511432360',
 '256588964152',
 '235622714832',
 '335524366801',
 '335511427393',
 '256574027160',
 '235031695203',
 '235677067106',
 '235529254095',
 '235741770603',
 '235180720856',
 '256081676427',
 '234374652511',
 '335398145876',
 '235560856693',
 '256202050181',
 '335397749637',
 '335356514587']

In [5]:
list_item_number

['335270737835',
 '252900220689',
 '232479816006',
 '232315264904',
 '334348192867',
 '235748987359',
 '235461387733',
 '255410646663',
 '235653299500',
 '256118799754',
 '256469386720',
 '256572886011',
 '255586148611',
 '235544392154',
 '233591979628',
 '335058229363',
 '255531675896',
 '233621258535',
 '256511996154',
 '235055850780',
 '256264372483',
 '235122553175',
 '253139036562',
 '235443868102',
 '335058230165',
 '254598959347',
 '256102649999',
 '335563408621',
 '332554123135',
 '252900019577',
 '235647995819',
 '335058269621',
 '256646743487',
 '255018389434',
 '334041377271',
 '256496980460',
 '334922305483',
 '256588980700',
 '332388018897',
 '235382407258',
 '232315321167',
 '234442824582',
 '256202118103',
 '335549164470',
 '234053013857',
 '334707355204',
 '256660158865',
 '232315259795',
 '335572848107',
 '335372891937',
 '234095593488',
 '235614466473',
 '232628030166',
 '235775365047',
 '335372826273',
 '256564411343',
 '235553119015',
 '232315242152',
 '235514544228

In [6]:
df_correct = pd.DataFrame({'No': range(1, len(list_item_number) + 1),
                           'Item Number': list_item_number})
df_correct.to_excel('./test_item_number.xlsx', index=False)

df_correct

,No,Item Number
0,1,335270737835
1,2,252900220689
2,3,232479816006
3,4,232315264904
4,5,334348192867
...,...,...
507,508,235584971081
508,509,256496796245
509,510,332371706664
510,511,256587735040


In [7]:
crawler_series['Request_Url'] = url_request

list_series_error.append(crawler_series)

list_series_error

[No                                                             1
 Url            https://www.ebay.com/sch/i.html?_dkr=1&iconV2R...
 Request_Url    https://www.ebay.com/sch/i.html?_dkr=1&iconV2R...
 Name: 0, dtype: object]

In [8]:
if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['No'],
                                                           ascending=[True],
                                                           ignore_index=True)
    df_error.to_excel('./test_item_number_error.xlsx', index=False)

df_error

,No,Url,Request_Url
0,1,https://www.ebay.com/sch/i.html?_dkr=1&iconV2R...,https://www.ebay.com/sch/i.html?_dkr=1&iconV2R...
